In [1]:
#Essentially a translator that takes your prompt and reformats for LangChain to better understand to process better
from langchain.adapters.openai import convert_openai_messages
from langchain_openai import ChatOpenAI
from tavily import TavilyClient
import os 

Tavily Search

In [2]:
client = TavilyClient(api_key=os.environ.get('TAVILY_API_KEY')) 
response = client.search(query= "Recommend me the top five fragrances")
#Returns sources sorted by relevant scores
print(response)

{'query': 'Recommend me the top five fragrances', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'These Are the Top 5 Fragrances Among NewBeauty Readers', 'url': 'https://www.newbeauty.com/top-5-favorite-fragrances-newbeauty-readers/', 'content': "Rounding out the top five is a tie between Maison Francis Kurkdijan's Baccarat Rouge 540 ($645) and Dior. Rouge 540 combines notes of wood, saffron, jasmine, and amber to create an unrivaled aroma.", 'score': 0.751375, 'raw_content': None}, {'title': 'The 31 Best Perfumes, Byrdie Editor-Tested', 'url': 'https://www.byrdie.com/best-perfumes-for-women-4770556', 'content': "| Top Notes: Tea, bergamot, Osmanthus | Cruelty-Free: No\nVersatile\nLong-lasting\nMay be overpowering to some\nByrdie / Jhett Thompson\nByrdie / Jessica Juliao\nByrdie / Jessica Juliao\nByrdie / Jessica Juliao\nByrdie / Jessica Juliao\nByrdie / Jessica Juliao\nByrdie / Jessica Juliao\nByrdie / Jhett Thompson\nBest for Date Night\nCarolina He

Processing & Displaying Results

In [3]:
for result in response['results']:
    print(result['title'])

These Are the Top 5 Fragrances Among NewBeauty Readers
The 31 Best Perfumes, Byrdie Editor-Tested
Best Fragrances of All Time: The W Editors' Favorite Perfumes
The 27 Very Best Perfumes - New York Magazine
FRAGRANTICA Readers' Choice AWARDS 2021


More Advanced Search Query

In [4]:
response = client.search(
    query="Based on this user’s preferences, what are the top 5 bestselling cologne/perfumes? Please provide the results in a single string separated by a ‘/’ (e.g., fragrance1/fragrance2/fragrance3/fragrance4/fragrance5): 'Fragrance Type: Cologne Fragrance Format(s): Eau de Parfum (EDP), Eau de Toilette (EDT), Solid Perfumes Fragrance Portfolio(s): Woody, Oriental, Spicy'",
    #A more in depth search but provides more comprehensive results
    search_depth='advanced', 
    max_results=1, 
    include_images=True, 
)
for result in response['results']:
    print(result['content'])

Home - Guides - Best Selling And Smelling Colognes For Men in 2025. Best Selling And Smelling Colognes For Men in 2025. April 11, ... This Oriental Woody fragrance comes in a gorgeous branded bottle that is synonymous with Tom Ford ... It is a woody spicy fragrance that mixes mint and blood orange at the head with a heart of spicy notes and


Quick Direct Search Query (Concise and Factual) Probably best for this project

In [5]:
print(client.qna_search(query= "Based on this user’s preferences, what are the top 5 bestselling cologne/perfumes? Please provide the results in a single string separated by a ‘/’ (e.g., fragrance1/fragrance2/fragrance3/fragrance4/fragrance5): 'Fragrance Type: Cologne Fragrance Format(s): Eau de Parfum (EDP), Eau de Toilette (EDT), Solid Perfumes Fragrance Portfolio(s): Woody, Oriental, Spicy'"))

Frédéric Malle Portrait Of A Lady/Calvin Klein Eternity for Men/Hermès H24/Byredo Mojave Ghost/Prada Ocean


Combining Tavily's search capabilities with LLMS using LangChain

In [6]:
#Essentially translates our query into a more formatted way for LangChain to understand
from langchain.adapters.openai import convert_openai_messages
#Allow us to interact with LLMS
from langchain_openai import ChatOpenAI


#Tavily Prompt (Search Query To Retrieve Data):
query = "Based on this user’s preferences, what are the top 5 bestselling cologne/perfumes? Please provide the results in a single string separated by a ‘/’ (e.g., fragrance1/fragrance2/fragrance3/fragrance4/fragrance5): 'Fragrance Type: Cologne Fragrance Format(s): Eau de Parfum (EDP), Eau de Toilette (EDT), Solid Perfumes Fragrance Portfolio(s): Woody, Oriental, Spicy'"
response = client.search(query, max_results= 10, search_depth="advanced")['results']
response

[{'title': '10 Best Woody Colognes For Men In 2025 - FashionBeans',
  'url': 'https://www.fashionbeans.com/article/best-woody-colognes/',
  'content': 'Discover the 10 best woody cologne to choose from. 155K; 99.3k; 629K; ... Best Spicy Woody Cologne: Paco Rabane 1 Million Intense ... Best Selling Woody Cologne: Gucci Guilty. View It On Amazon .',
  'score': 0.77179235,
  'raw_content': None},
 {'title': 'Best-Selling Branded Perfumes in 2025 - frenchfragrance.com',
  'url': 'https://frenchfragrance.com/blog/best-selling-branded-perfumes-in-2025',
  'content': 'Top Women Perfumes Perfumes Women Perfumes Eyes Eye Cream Eye Cream Hair Perfume Perfumes Perfumes French Fragrance offers you a detailed look at some of the best-selling and most iconic perfumes in the world. Their strong brand identities and enduring appeal highlight the diversity of popular scents in the fragrance world. Elie Saab Le Parfum For Women is a luxurious fragrance that blends confidence and charm with bold sophisti

In [7]:
#LangChain Prompt (Search Query To Process Data): 
prompt = [
    #Define the AI's job description
    {
        'role':'system', 
        'content': f''' 
        You are an AI critical thinker research assistant. 
        Your purpose is to help recommend user's best selling fragrances.
        '''
    },
    #Give Specific instructions for the current task based on the input data
    {
        'role': 'user', 
        'content': f'''Information: """{response}"""
        Using the information above, answer the following query: """{query}""". 
        '''
    }
]

lc_messages = convert_openai_messages(prompt)
lc_messages

[SystemMessage(content=" \n        You are an AI critical thinker research assistant. \n        Your purpose is to help recommend user's best selling fragrances.\n        ", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Information: """[{\'title\': \'10 Best Woody Colognes For Men In 2025 - FashionBeans\', \'url\': \'https://www.fashionbeans.com/article/best-woody-colognes/\', \'content\': \'Discover the 10 best woody cologne to choose from. 155K; 99.3k; 629K; ... Best Spicy Woody Cologne: Paco Rabane 1 Million Intense ... Best Selling Woody Cologne: Gucci Guilty. View It On Amazon .\', \'score\': 0.77179235, \'raw_content\': None}, {\'title\': \'Best-Selling Branded Perfumes in 2025 - frenchfragrance.com\', \'url\': \'https://frenchfragrance.com/blog/best-selling-branded-perfumes-in-2025\', \'content\': \'Top Women Perfumes Perfumes Women Perfumes Eyes Eye Cream Eye Cream Hair Perfume Perfumes Perfumes French Fragrance offers you a detailed look at some of the be

In [8]:
response = ChatOpenAI(model='gpt-4o-mini').invoke(lc_messages)
print(response.content)

Gucci Guilty/Paco Rabane 1 Million Intense/Frédéric Malle Portrait Of A Lady/Calvin Klein Eternity for Men/Hermès H24
